In [4]:
import numpy as np
import pandas as pd
import redis
import cv2

In [5]:
r = redis.StrictRedis(
    host="redis-11755.c308.sa-east-1-1.ec2.cloud.redislabs.com", 
    port=11755,
    username="default",
    password="CzdAM8ReaYrKSySrvShim7abK4ZENraC",
)

In [6]:
r.ping()

True

In [31]:
r.flushall()

True

In [7]:
# load the data (npzip)
np_file = np.load('dataframe_students_teachers.npz', allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values, columns=col_names)
df

,Name,Role,Facial_Features
0,Angelina Jolie,student,"[0.19881934, 1.4748836, -0.0979588, -0.8805054..."
1,Barack Obama,teacher,"[-0.22177827, 1.5762169, -0.50307816, 0.136012..."
2,Chris Evans,student,"[0.35691044, -0.41491055, 0.03921845, -0.15436..."
3,Morgan Freeman,teacher,"[0.7627893, 0.89061487, 0.46873555, 0.25898102..."
4,Scarlett Johansson,student,"[-0.13535985, -0.2821385, -0.4270084, -0.28251..."


In [8]:
df['name_role'] = df['Name']+'@'+df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,student,"[0.19881934, 1.4748836, -0.0979588, -0.8805054...",Angelina Jolie@student
1,Barack Obama,teacher,"[-0.22177827, 1.5762169, -0.50307816, 0.136012...",Barack Obama@teacher
2,Chris Evans,student,"[0.35691044, -0.41491055, 0.03921845, -0.15436...",Chris Evans@student
3,Morgan Freeman,teacher,"[0.7627893, 0.89061487, 0.46873555, 0.25898102...",Morgan Freeman@teacher
4,Scarlett Johansson,student,"[-0.13535985, -0.2821385, -0.4270084, -0.28251...",Scarlett Johansson@student


In [9]:
records = df[['name_role', 'Facial_Features']].to_dict(orient='records')
len(records)

5

### Save data in redis
* redis hashes
* keyname = 'academy:register'

In [32]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial_Features']

    # convert np array into bytes
    vector_bytes = vector.tobytes()
    r.hset('academy:register', key=name_role, value=vector_bytes)

In [11]:
vector.dtype

dtype('float32')

### Retrieve records from redis

In [12]:
retrieved_dict = r.hgetall(name='academy:register')
retrieved_series = pd.Series(retrieved_dict)
retrieved_series

b'Angelina Jolie@student'        b'L\x97K>\xfc\xc8\xbc?\xa0\x9e\xc8\xbd\xcdha\x...
b'Chris Evans@student'           b'\xf7\xbc\xb6>(o\xd4\xbe\x87\xa3 =\xbb\x11\x1...
b'Gugubs@Student'                b'\xe4\x8d\xba<:\x0b\xee<\xba\x02)=\xd4\xf1>=\...
b'Scarlett Johansson@student'    b'\xc6\x9b\n\xbeut\x90\xbe\xd8\xa0\xda\xbe\x17...
b'Morgan Freeman@teacher'        b')FC?V\xffc?\x1b\xfe\xef>)\x99\x84>\xee\xfcb\...
b'Barack Obama@teacher'          b'\xd8\x19c\xbez\xc1\xc9?\xbb\xc9\x00\xbf\xf2F...
dtype: object

In [13]:
#np.frombuffer(retrive_series[b'Morgan Freeman@Teacher'],dtype=np.float32).shape
retrived_series = retrieved_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrieved_series.index
index = list(map(lambda x: x.decode(), index))

In [14]:
retrieved_series.index = index

In [15]:
retrive_df = retrived_series.to_frame().reset_index()
retrive_df.columns = ['name_role','facial_features']
retrive_df

,name_role,facial_features
0,b'Angelina Jolie@student',"[0.19881934, 1.4748836, -0.0979588, -0.8805054..."
1,b'Chris Evans@student',"[0.35691044, -0.41491055, 0.03921845, -0.15436..."
2,b'Gugubs@Student',"[0.022772737, 0.029058088, 0.041262366, 0.0466..."
3,b'Scarlett Johansson@student',"[-0.13535985, -0.2821385, -0.4270084, -0.28251..."
4,b'Morgan Freeman@teacher',"[0.7627893, 0.89061487, 0.46873555, 0.25898102..."
5,b'Barack Obama@teacher',"[-0.22177827, 1.5762169, -0.50307816, 0.136012..."


# Registration form

In [16]:
import cv2
from insightface.app import FaceAnalysis

In [17]:
#config face analysis
faceapp = FaceAnalysis(name='buffalo_sc',
                       root='models',
                       providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

faceapp.prepare(ctx_id=0,
                det_size=(640, 640),
                det_thresh=0.5)

/home/glccampos/Downloads/pass/envs/face_reco/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_sc/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_sc/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_sc/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_sc/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_sc/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


### 1st step collect person role and name

In [25]:
# py input
person_name = input('Enter name: ')
trials = 3
for i in range(trials):
    role = input("""
    Please choose:
                1. Student
                2. Teacher

                Enter either 1 or 2
                """)
    if role in ('1', '2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        break
    else:
        print('Invalid entry')
        if i == 3:
            print('Exceed max attempts!')

key = person_name + '@' + role
print('Your name =', person_name)
print('Your role', role)
print('key =', key)

Your name = gugubs
Your role Student
key = gugubs@Student


In [26]:
person_name, role

('gugubs', 'Student')

### 2nd step collect samples

In [27]:
cap = cv2.VideoCapture(0)
face_embeddings = []
samples = 0

while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
    # get results from insightface model
    results = faceapp.get(frame, max_num=1)
    for res in results:
        samples += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 1)

        #facial embeddings
        embeddings = res['embedding']
        face_embeddings.append(embeddings)
    
    if samples >= 200:
        break

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### 3rd step optimize data and save in redis

In [28]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [29]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [30]:
# save key and facial bytes into redis db
r.hset(name='academy:register', key=key, value=x_mean_bytes)

1